Data cleaning
#drop unnamed columns
#convert and parse dates and times
#handle nulls (1 value)
#handle 'No info'
#handle duplicates
#categorical encoding (airline, source, destination) #Ali
#rename columns

EDA
#data viz #khalid

Feature and target engineering

ML modeling

Conclusion --perform testing


In [1]:
# Helper packages
import missingno as msno
import numpy as np
import pandas as pd
from plotnine import ggplot, aes, geom_density, geom_line, geom_point, ggtitle

# Modeling pre-processing with scikit-learn functionality
from sklearn.model_selection import train_test_split
from sklearn.compose import TransformedTargetRegressor
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA

# Modeling pre-processing with non-scikit-learn packages
from category_encoders.ordinal import OrdinalEncoder
from feature_engine.encoding import RareLabelEncoder

# Modeling
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_excel('flights.xlsx')

In [3]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional Info,Price,Unnamed: 11,Unnamed: 12
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3898,NaN,NaN
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7663,NaN,NaN
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13883,NaN,NaN
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6219,NaN,NaN
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13303,NaN,NaN


In [4]:
##drop unnamed columns
df.drop(['Unnamed: 11'], axis =1, inplace=True)

In [5]:
df.drop(['Unnamed: 12'], axis=1, inplace=True)

In [6]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3898
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7663
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13883
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6219
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13303


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [8]:
###convert time to DateTime type
df['Date_of_Journey']=pd.to_datetime(df['Date_of_Journey'])
df['Dep_Time']=pd.to_datetime(df['Dep_Time'])
df['Arrival_Time']=pd.to_datetime(df['Arrival_Time'])

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Airline          10683 non-null  object        
 1   Date_of_Journey  10683 non-null  datetime64[ns]
 2   Source           10683 non-null  object        
 3   Destination      10683 non-null  object        
 4   Route            10682 non-null  object        
 5   Dep_Time         10683 non-null  datetime64[ns]
 6   Arrival_Time     10683 non-null  datetime64[ns]
 7   Duration         10683 non-null  object        
 8   Total_Stops      10682 non-null  object        
 9   Additional Info  10683 non-null  object        
 10  Price            10683 non-null  int64         
dtypes: datetime64[ns](3), int64(1), object(7)
memory usage: 918.2+ KB


In [10]:
#devide years, months, days of datetime dtype
df['journey_year']=df['Date_of_Journey'].dt.year
df['journey_month']=df['Date_of_Journey'].dt.month
df['journey_day']=df['Date_of_Journey'].dt.day

In [11]:
#drop Date_of_Journey
df.drop(['Date_of_Journey'], axis=1, inplace=True)

In [12]:
#devide dep_time to two columns: hr and min 
df['dep_hour']=df['Dep_Time'].dt.hour
df['dep_min']=df['Dep_Time'].dt.minute

In [13]:
#drop Dep_time
df.drop(['Dep_Time'], axis=1, inplace=True)

In [14]:
###devide arrival date
df['arrival_year']=df['Arrival_Time'].dt.year
df['arrival_month']=df['Arrival_Time'].dt.month
df['arrival_day']=df['Arrival_Time'].dt.day

In [15]:
##devide arrival time 
df['arrival_hour']=df['Arrival_Time'].dt.hour
df['arrival_min']=df['Arrival_Time'].dt.minute

In [16]:
df.head()

,Airline,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional Info,Price,journey_year,journey_month,journey_day,dep_hour,dep_min,arrival_year,arrival_month,arrival_day,arrival_hour,arrival_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2022-03-22 01:10:00,2h 50m,non-stop,No info,3898,2019,3,24,22,20,2022,3,22,1,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2022-02-14 13:15:00,7h 25m,2 stops,No info,7663,2019,1,5,5,50,2022,2,14,13,15
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2022-06-10 04:25:00,19h,2 stops,No info,13883,2019,9,6,9,25,2022,6,10,4,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,2022-02-14 23:30:00,5h 25m,1 stop,No info,6219,2019,12,5,18,5,2022,2,14,23,30
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,2022-02-14 21:35:00,4h 45m,1 stop,No info,13303,2019,1,3,16,50,2022,2,14,21,35


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Airline          10683 non-null  object        
 1   Source           10683 non-null  object        
 2   Destination      10683 non-null  object        
 3   Route            10682 non-null  object        
 4   Arrival_Time     10683 non-null  datetime64[ns]
 5   Duration         10683 non-null  object        
 6   Total_Stops      10682 non-null  object        
 7   Additional Info  10683 non-null  object        
 8   Price            10683 non-null  int64         
 9   journey_year     10683 non-null  int64         
 10  journey_month    10683 non-null  int64         
 11  journey_day      10683 non-null  int64         
 12  dep_hour         10683 non-null  int64         
 13  dep_min          10683 non-null  int64         
 14  arrival_year     10683 non-null  int64

In [18]:
df.isnull().sum()

Airline            0
Source             0
Destination        0
Route              1
Arrival_Time       0
Duration           0
Total_Stops        1
Additional Info    0
Price              0
journey_year       0
journey_month      0
journey_day        0
dep_hour           0
dep_min            0
arrival_year       0
arrival_month      0
arrival_day        0
arrival_hour       0
arrival_min        0
dtype: int64

In [19]:
df1 = df[df.isna().any(axis=1)]
df1

,Airline,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional Info,Price,journey_year,journey_month,journey_day,dep_hour,dep_min,arrival_year,arrival_month,arrival_day,arrival_hour,arrival_min
9039,Air India,Delhi,Cochin,NaN,2022-05-07 09:25:00,23h 40m,NaN,No info,7481,2019,6,5,9,45,2022,5,7,9,25


In [20]:
#divide duration to two columns hours and mins
duration = list(df["Duration"]) #list Duration so we can spply split() and refrenec it 
duration_hours = [] #create empty lists to append later
duration_mins = []
for i in range(len(duration)):
    duration_hours.append(duration[i].split(sep = "h")[0])
    duration_mins.append(duration[i].split(sep = "m")[0].split()[-1])

In [21]:
##adding duration_hours and duration_mins to df
df['duration_hours']=duration_hours
df['duration_mins']=duration_mins

In [22]:
#drop Duration
df.drop(['Duration'], axis=1, inplace=True)

In [23]:
df.head()

,Airline,Source,Destination,Route,Arrival_Time,Total_Stops,Additional Info,Price,journey_year,journey_month,journey_day,dep_hour,dep_min,arrival_year,arrival_month,arrival_day,arrival_hour,arrival_min,duration_hours,duration_mins
0,IndiGo,Banglore,New Delhi,BLR → DEL,2022-03-22 01:10:00,non-stop,No info,3898,2019,3,24,22,20,2022,3,22,1,10,2,50
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2022-02-14 13:15:00,2 stops,No info,7663,2019,1,5,5,50,2022,2,14,13,15,7,25
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2022-06-10 04:25:00,2 stops,No info,13883,2019,9,6,9,25,2022,6,10,4,25,19,19h
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,2022-02-14 23:30:00,1 stop,No info,6219,2019,12,5,18,5,2022,2,14,23,30,5,25
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,2022-02-14 21:35:00,1 stop,No info,13303,2019,1,3,16,50,2022,2,14,21,35,4,45


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Airline          10683 non-null  object        
 1   Source           10683 non-null  object        
 2   Destination      10683 non-null  object        
 3   Route            10682 non-null  object        
 4   Arrival_Time     10683 non-null  datetime64[ns]
 5   Total_Stops      10682 non-null  object        
 6   Additional Info  10683 non-null  object        
 7   Price            10683 non-null  int64         
 8   journey_year     10683 non-null  int64         
 9   journey_month    10683 non-null  int64         
 10  journey_day      10683 non-null  int64         
 11  dep_hour         10683 non-null  int64         
 12  dep_min          10683 non-null  int64         
 13  arrival_year     10683 non-null  int64         
 14  arrival_month    10683 non-null  int64

In [45]:
df.rename(columns={'Additional Info':'additional_info'}, inplace=True)

In [26]:
###Impute nulls

In [27]:
#categorical encoding (airline, source, destination, )
#impute null (1 value)
#rename columns
#check duplicates

In [28]:
###encode stops
df['Total_Stops'].replace({'non-stop':0, '1 stop': 1, '2 stops': 2, '3 stops':3, '4 stops':4}, inplace=True)

In [46]:
df.columns

Index(['Airline', 'Source', 'Destination', 'Route', 'Arrival_Time',
       'Total_Stops', 'additional_info', 'Price', 'journey_year',
       'journey_month', 'journey_day', 'dep_hour', 'dep_min', 'arrival_year',
       'arrival_month', 'arrival_day', 'arrival_hour', 'arrival_min',
       'duration_hours', 'duration_mins'],
      dtype='object')

In [47]:
df.additional_info.value_counts()

No info                         8345
In-flight meal not included     1982
No check-in baggage included     320
1 Long layover                    19
Change airports                    7
Business class                     4
No Info                            3
2 Long layover                     1
1 Short layover                    1
Red-eye flight                     1
Name: additional_info, dtype: int64

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Airline          10683 non-null  object        
 1   Source           10683 non-null  object        
 2   Destination      10683 non-null  object        
 3   Route            10682 non-null  object        
 4   Arrival_Time     10683 non-null  datetime64[ns]
 5   Total_Stops      10682 non-null  float64       
 6   Additional Info  10683 non-null  object        
 7   Price            10683 non-null  int64         
 8   journey_year     10683 non-null  int64         
 9   journey_month    10683 non-null  int64         
 10  journey_day      10683 non-null  int64         
 11  dep_hour         10683 non-null  int64         
 12  dep_min          10683 non-null  int64         
 13  arrival_year     10683 non-null  int64         
 14  arrival_month    10683 non-null  int64

In [ ]:
df.ad

In [31]:
df.nunique()

Airline              12
Source                5
Destination           6
Route               128
Arrival_Time       1343
Total_Stops           5
Additional Info      10
Price              1870
journey_year          1
journey_month         7
journey_day           9
dep_hour             24
dep_min              12
arrival_year          1
arrival_month         5
arrival_day          20
arrival_hour         24
arrival_min          12
duration_hours       44
duration_mins        40
dtype: int64